# Architecture 1: Agent Conversationnel Simple (Baseline)

Cette architecture implémente un agent conversationnel basé sur un LLM (Llama 3.2 3B) spécialement fine-tuned avec LoRA sur les données conversationnelles d'EasyTransfert.

## Caractéristiques principales:
- **Modèle**: Llama 3.2 3B Instruct
- **Technique**: LoRA (Low-Rank Adaptation) avec rang=16, alpha=32
- **Framework**: Unsloth pour entraînement optimisé
- **Données**: 3000+ conversations historiques EasyTransfert
- **Approche**: Toutes les connaissances encodées dans les paramètres du modèle

<a href="https://colab.research.google.com/github/AmedBah/memoire/blob/main/notebooks/architecture_1/01_architecture_1_simple_agent_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🚀 Configuration pour Google Colab

**Note**: Cette section est spécifique à Google Colab. Si vous exécutez ce notebook localement, vous pouvez ignorer ces cellules.

In [ ]:
# Vérifier le type de runtime
import os
import sys

# Détecter si on est sur Colab
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    print("✓ Exécution sur Google Colab")
    
    # Vérifier le type de GPU
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✓ GPU détecté: {gpu_name}")
        print(f"✓ Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        
        # Recommandations selon le GPU
        if 'T4' in gpu_name:
            print("\n⚠️  GPU T4 détecté (16 GB): Convient pour ce notebook mais les temps d'entraînement seront plus longs")
        elif 'V100' in gpu_name or 'A100' in gpu_name:
            print(f"\n✓ {gpu_name}: Parfait pour ce notebook!")
    else:
        print("\n❌ ATTENTION: Aucun GPU détecté!")
        print("   Pour activer le GPU: Runtime > Change runtime type > GPU")
        print("   Pour Colab Pro: Choisir 'High-RAM' ou 'Premium GPU'")
else:
    print("✓ Exécution locale")
    import torch
    if torch.cuda.is_available():
        print(f"✓ GPU disponible: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️  Aucun GPU détecté - l'entraînement sera très lent")

In [ ]:
# Monter Google Drive (uniquement sur Colab)
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Définir le chemin vers les données
    # OPTION 1: Données dans Google Drive
    # DATA_DIR = '/content/drive/MyDrive/memoire/data'
    
    # OPTION 2: Cloner le repo et utiliser les données locales
    print("\n📥 Clonage du repository...")
    !git clone https://github.com/AmedBah/memoire.git /content/memoire
    DATA_DIR = '/content/memoire/data'
    
    print(f"\n✓ Répertoire de données: {DATA_DIR}")
    
    # Vérifier que les données sont présentes
    if os.path.exists(DATA_DIR):
        print("✓ Données trouvées!")
        !ls -lh {DATA_DIR}
    else:
        print(f"❌ ERREUR: Répertoire {DATA_DIR} non trouvé!")
        print("   Veuillez soit:")
        print("   1. Copier le dossier 'data' dans votre Google Drive")
        print("   2. Ou le repository sera cloné automatiquement")
else:
    # Exécution locale
    DATA_DIR = '../../data'
    print(f"✓ Répertoire de données local: {DATA_DIR}")

In [ ]:
# Fonction helper pour obtenir les chemins de données
def get_data_path(relative_path):
    """Obtenir le chemin absolu d'un fichier de données"""
    return os.path.join(DATA_DIR, relative_path)

# Exemples de chemins
print("Chemins de données configurés:")
print(f"  Conversations: {get_data_path('conversations/conversation_1000_finetune.jsonl')}")
print(f"  FAQs: {get_data_path('faqs/faq_easytransfert.json')}")
print(f"  Opérateurs: {get_data_path('operators/operators_info.json')}")
print(f"  Procédures: {get_data_path('procedures/procedures_resolution.json')}")
print(f"  Expressions: {get_data_path('expressions/expressions_ivoiriennes.json')}")
print(f"  Documents: {get_data_path('documents/doc.txt.txt')}")

## 1. Installation et Configuration

In [ ]:
# Installation des dépendances
!pip install -q unsloth transformers datasets accelerate peft bitsandbytes trl wandb

import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import wandb
import json
import os

print(f"GPU disponible: {torch.cuda.is_available()}")
print(f"Nombre de GPUs: {torch.cuda.device_count()}")

## 2. Configuration du Modèle

In [ ]:
# Configuration
max_seq_length = 2048  # Longueur maximale de séquence
dtype = None  # Auto-détection (Float16 pour Tesla T4, V100, Bfloat16 pour Ampere+)
load_in_4bit = True  # Quantification 4-bit pour économiser la mémoire

# Chargement du modèle avec Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    trust_remote_code=True,
)

print("Modèle chargé avec succès!")

## 3. Configuration LoRA

Configuration selon les spécifications de l'architecture:
- Rang (r): 16 - Dimension des matrices de faible rang
- Alpha (α): 32 - Facteur de mise à l'échelle
- Dropout: 0.05 - Régularisation
- Modules cibles: Query/Key/Value des couches d'attention

In [ ]:
# Configuration LoRA selon l'architecture définie
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rang des matrices de faible rang
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,  # Facteur de mise à l'échelle
    lora_dropout=0.05,  # Régularisation
    bias="none",
    use_gradient_checkpointing="unsloth",  # Optimisation mémoire
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# Afficher les paramètres entraînables (~1% du total, ~30M sur 3B)
trainable, total = model.get_nb_trainable_parameters()
print(f"Paramètres entraînables: {trainable:,} / {total:,} ({100 * trainable / total:.2f}%)")

## 4. Préparation des Données

Chargement et formatage des conversations EasyTransfert selon le template Llama 3 Chat

In [ ]:
# Template système pour EasyTransfert
system_prompt = """Tu es un assistant intelligent du service client EasyTransfert, une application de transfert d'argent mobile en Côte d'Ivoire.

Règles importantes:
- Mémorisation obligatoire: Ne jamais redemander une information déjà fournie
- Formats d'identifiants: 
  * EasyTransfert: EFB.*
  * Orange envoi: MP*
  * MTN: chiffres uniquement
  * Moov envoi: MRCH*/CF*
  * Wave: format variable (souvent T*)
- Ton chaleureux: Utilise des émojis appropriés 🤗😊
- Mention régulière d'EasyTransfert
- Pas de répétition des salutations
- Adaptation au niveau d'urgence
- Contact service client: 2522018730 (disponible 24h/24 via WhatsApp)

Opérateurs supportés: MTN, Orange, Moov, Wave, Trésor Money"""

# Template de conversation Llama 3
alpaca_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}<|eot_id|>"""

# Fonction de formatage des conversations
def format_conversation(messages):
    """Formate une conversation multi-tours en prompt Llama 3"""
    formatted_messages = []
    
    # Construire la conversation
    conversation = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
    conversation += system_prompt + "<|eot_id|>"
    
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        
        conversation += f"<|start_header_id|>{role}<|end_header_id|>\n\n{content}<|eot_id|>"
    
    return {"text": conversation}

# Charger les données de conversation
dataset = load_dataset("json", data_files="get_data_path('conversations/conversation_1000_finetune.jsonl')", split="train")
print(f"Nombre de conversations: {len(dataset)}")

# Formater les conversations
formatted_dataset = dataset.map(
    lambda x: format_conversation(x["messages"]),
    remove_columns=dataset.column_names
)

print("\nExemple de conversation formatée:")
print(formatted_dataset[0]["text"][:500] + "...")

## 5. Configuration de l'Entraînement

In [ ]:
# Initialiser Weights & Biases pour le suivi
wandb.init(
    project="easytransfert-architecture1",
    name="llama-3.2-3b-lora-finetuning",
    config={
        "model": "Llama-3.2-3B-Instruct",
        "lora_r": 16,
        "lora_alpha": 32,
        "lora_dropout": 0.05,
        "max_seq_length": max_seq_length,
    }
)

# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./outputs_arch1",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=3407,
    save_strategy="steps",
    save_steps=100,
    report_to="wandb",
)

print("Configuration d'entraînement définie.")

## 6. Entraînement du Modèle

Entraînement uniquement sur les réponses de l'assistant (train_on_responses_only)

In [ ]:
from unsloth.chat_templates import train_on_responses_only

# Créer le trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

# Entraîner uniquement sur les réponses de l'assistant
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)

print("Trainer configuré. Début de l'entraînement...")

# Lancer l'entraînement
trainer_stats = trainer.train()

print("\nEntraînement terminé!")
print(f"Temps d'entraînement: {trainer_stats.metrics['train_runtime']:.2f} secondes")
print(f"Loss finale: {trainer_stats.metrics['train_loss']:.4f}")

## 7. Sauvegarde du Modèle

In [ ]:
# Sauvegarder le modèle fine-tuné
model.save_pretrained("./models/architecture1_lora")
tokenizer.save_pretrained("./models/architecture1_lora")

print("Modèle sauvegardé dans ./models/architecture1_lora")

# Sauvegarder aussi en format GGUF pour déploiement (optionnel)
# model.save_pretrained_gguf("architecture1_model", tokenizer, quantization_method="q4_k_m")

wandb.finish()

## 8. Inférence et Test

In [ ]:
# Activer le mode inférence rapide
FastLanguageModel.for_inference(model)

def chat(user_message, conversation_history=[]):
    """Fonction de chat avec le modèle fine-tuné"""
    
    # Construire le prompt avec l'historique
    messages = conversation_history + [{"role": "user", "content": user_message}]
    
    prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
    prompt += system_prompt + "<|eot_id|>"
    
    for msg in messages:
        prompt += f"<|start_header_id|>{msg['role']}<|end_header_id|>\n\n{msg['content']}<|eot_id|>"
    
    prompt += "<|start_header_id|>assistant<|end_header_id|>\n\n"
    
    # Générer la réponse
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        min_p=0.1,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    
    return response.strip()

# Tests
print("=" * 80)
print("Test 1: Salutation initiale")
print("=" * 80)
response1 = chat("Bonjour")
print(f"Assistant: {response1}\n")

print("=" * 80)
print("Test 2: Problème de transaction")
print("=" * 80)
conversation = [{"role": "user", "content": "Bonjour"}, 
                {"role": "assistant", "content": response1}]
response2 = chat("Mon argent n'est pas arrivé", conversation)
print(f"Assistant: {response2}\n")

print("=" * 80)
print("Test 3: Question sur les opérateurs")
print("=" * 80)
response3 = chat("Quels sont les opérateurs supportés par EasyTransfert ?")
print(f"Assistant: {response3}\n")

## 9. Évaluation Quantitative

Métriques de performance de l'Architecture 1

In [ ]:
import time
import numpy as np

def evaluate_model(test_queries, num_runs=5):
    """Évaluation des performances du modèle"""
    
    latencies = []
    response_lengths = []
    
    for query in test_queries:
        for _ in range(num_runs):
            start_time = time.time()
            response = chat(query)
            latency = time.time() - start_time
            
            latencies.append(latency)
            response_lengths.append(len(response.split()))
    
    print("Métriques de Performance - Architecture 1")
    print("=" * 60)
    print(f"Latence moyenne: {np.mean(latencies):.2f}s (±{np.std(latencies):.2f}s)")
    print(f"Latence médiane: {np.median(latencies):.2f}s")
    print(f"Latence min/max: {np.min(latencies):.2f}s / {np.max(latencies):.2f}s")
    print(f"Longueur moyenne réponse: {np.mean(response_lengths):.1f} mots")
    print("=" * 60)
    
    return {
        "latency_mean": np.mean(latencies),
        "latency_std": np.std(latencies),
        "latency_median": np.median(latencies),
        "response_length_mean": np.mean(response_lengths)
    }

# Requêtes de test
test_queries = [
    "Bonjour",
    "Mon transfert n'est pas arrivé",
    "Comment faire un transfert de MTN vers Orange ?",
    "J'ai oublié mon mot de passe",
    "Quels sont les frais de transaction ?"
]

metrics = evaluate_model(test_queries, num_runs=3)

# Logger les métriques dans WandB
wandb.log(metrics)

## 10. Avantages et Limites de l'Architecture 1

### Avantages:
- ✅ **Simplicité**: Architecture directe sans dépendances externes
- ✅ **Efficacité**: Inférence rapide (~2-3s par réponse)
- ✅ **Mémoire**: Adaptateurs LoRA légers (~50 MB)
- ✅ **Déploiement**: Facile sur GPU grand public
- ✅ **Cohérence**: Style conversationnel homogène

### Limites:
- ❌ **Hallucinations**: Risque d'inventer des informations
- ❌ **Actualisation**: Nécessite réentraînement pour nouvelles données
- ❌ **Traçabilité**: Pas de citation de sources
- ❌ **Connaissances**: Limitées aux données d'entraînement
- ❌ **Complexité**: Difficultés avec requêtes multi-étapes